1.	Install/import the libraries

In [ ]:
import torch
import pandas as pd

!pip install transformers
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

2.	Load the dataset


In [ ]:
df = pd.read_csv('dataset.csv',encoding="utf-8") # The data should have two columns: occupation and kldb.


Division of the kldb code for training with separate digits

In [ ]:
def kldb_devision(_data, _start_digit,_end_digit):
  _data['col'] = _data['kldb'].astype(str)
  _data['kldb_new'] = _data['col'].str[_start_digit:_end_digit]


In [ ]:
df.kldb = kldb_devision(df,0,1)

In [ ]:
# df['ctotal'] = df.cberuf.astype(str).str.cat(df.chelp1.astype(str), sep=', ')
# df['ctotal1'] = df.ctotal.astype(str).str.cat(df.chelp2.astype(str), sep=', ')
# df['ctotal2'] = df.ctotal1.astype(str).str.cat(df.chelp3.astype(str), sep=', ')
# df['ctotal3'] = df.ctotal2.astype(str).str.cat(df.chelp4.astype(str), sep=', ')
# df['ctotal4'] = df.ctotal3.astype(str).str.cat(df.chelp5.astype(str), sep=', ')
# # # # df['cberuf_cfunktion'] = df.cberuf_cfunktion_cfunktion.astype(str).str.cat(df.cfunkti2.astype(str), sep=', ')
# # # # df.cberuf_cfunktion[0]
# df.cberuf=df.ctotal4

In [ ]:
seq_len = [len(i.split()) for i in df.occupation]
max(seq_len)

In [ ]:
df['kldb'].value_counts()

Encoding the Labels

In [ ]:
possible_labels = df.kldb.unique()  
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index 
label_dict                                        

In [ ]:
df['label'] = df.kldb.replace(label_dict)

3.	Split it into three sets – train, validation, and test (train and validation set for fine-tuning, test set for predictions)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.0001, 
                                                  random_state=1)
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.1, random_state=1)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.loc[X_test, 'data_type'] = 'test'



df.groupby(['data_type']).count()   


4.	Import BERT Model and BERT Tokenizer & tokenize the text

In [ ]:
tokenizer = BertTokenizer.from_pretrained('deepset/gbert-base', 
                                         do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus( 
    df[df.data_type=='train'].occupation.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].occupation.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'


)
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].occupation.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

#train
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)
#input_id_x = df[df.data_type=='train'].p_lfdnr.values
#input_id_x = df[df.data_type=='train'].col.values

#validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

#test
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Downloading:   0%|          | 0.00/240k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/362 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2306: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
df[df.data_type=='val'].count

5. Define model architecture

In [ ]:
model = BertForSequenceClassification.from_pretrained("deepset/gbert-base",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
len(label_dict)


6. Prepare the dataloaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16  #It must be adjusted based on sequence length. 

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,    
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


dataloader_test = DataLoader(dataset_test,    
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)
 

7. Specify the optimizer and scheduler

Help: https://huggingface.co/docs/transformers/main_classes/optimizer_schedules 

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup,Adafactor

optimizer = AdamW(model.parameters(),
                 lr=1e-5)
                  # eps=1e-8,
                  # betas= (0.9,0.99),
                  # weight_decay = 0.1)
                  
epochs = 5         # #It must be adjusted based on data

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


8. Fine-Tune (train) BERT

In [ ]:
input_val=df[df.data_type=='val'].occupation.values

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Metrics

In [ ]:
from sklearn.metrics import f1_score,recall_score,precision_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def recall_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat, preds_flat, average='weighted')

def precision_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return precision_score(labels_flat, preds_flat, average='weighted')
    


Training loop

In [ ]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals= [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        #berufe = inputs['input_ids'].cpu().numpy()
        #kldbs = inputs['col'].cpu().numpy()  
        
        predictions.append(logits)
        true_vals.append(label_ids)
        #inputData.append(berufe)
        #input_kldb.append(kldbs)
    
    loss_val_avg = loss_val_total/len(dataloader_val)
     
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    inputData= np.concatenate(inputData, axis=0)
    input_kldb = np.concatenate(input_kldb, axis = 0 )
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'0{epoch}.model')

        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals= evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_recall = recall_score_func(predictions, true_vals)
    val_precision = precision_score_func(predictions, true_vals)
 
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Recall (Weighted): {val_recall}')
    tqdm.write(f'Precision (Weighted): {val_precision}')

    

Add and pre-process test set

In [ ]:
import pandas as pd
test = pd.read_csv('TEST.csv')


In [ ]:
test.kldb = kldb_devision(test,0,1)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('deepset/gbert-base', 
                                         do_lower_case=True)
X_data = test.index.values;
test.loc[X_data, 'data_type'] = 'test'
encoded_data = tokenizer.batch_encode_plus(
    test[test.data_type=='test'].cberuf.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)
  
input_ids_data = encoded_data['input_ids']
attention_masks_data = encoded_data['attention_mask']

dataset_data = TensorDataset(input_ids_data, attention_masks_data)


Using the saved model to code the test set

In [ ]:
import torch
from torch import nn
from transformers import Trainer

model = BertForSequenceClassification.from_pretrained("deepset/gbert-base",
                                                      num_labels=(1020),                                             
                                                      output_attentions=False,
                                                      output_hidden_states=False)
                                                      #ignore_mismatched_sizes=True)
model.load_state_dict(torch.load('08.model', map_location=torch.device('cpu')))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)



In [ ]:
dataloader_input = DataLoader(dataset_data,
                                sampler=SequentialSampler(dataset_data),
                                batch_size=16)


In [ ]:
import numpy as np

def classificate(_data):

    model.eval()
    
    predictions, inputData = [], []
    
    for batch in _data:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1]
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
      
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        berufe = inputs['input_ids'].cpu().numpy() 
        
        predictions.append(logits)
        inputData.append(berufe)
    
    
    predictions = np.concatenate(predictions, axis=0)
    inputData= np.concatenate(inputData, axis=0)
            
    return  predictions,inputData
predictions,inputData = classificate(dataloader_input)

In [ ]:
df['kldb'].value_counts()
possible_labels = df.kldb.unique()   

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index 
possible_labels      

In [ ]:
def getLabel(_data):
  label_dict_inverse = {v: k for k, v in label_dict.items()}
  arr=[]
  for x in _data:
    arr.append(label_dict_inverse[x])
  return arr
def softmax(y_preds):
  softM = np.exp(y_preds)/np.sum(np.exp(y_preds),axis=0)
  return softM  

def get_data_and_label(preds,beruf):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    

    preds_flat = np.argmax(preds, axis=1).flatten()
    beruf_flat = beruf.flatten()
    input_val=test.cberuf.values
    lastKLDB = test.predictions.values
    input_true_value = test.kldb.values
    input_id = test.p_lfdnr.values
    iput_all_kldb= test['col']
 
 
    arr = []
    result = []
    acc =[]
    

    for label in np.unique(preds_flat):
        y_preds = preds_flat[preds_flat==label]
        y_beruf = input_val[preds_flat==label]
        y_lastKldb= lastKLDB[preds_flat==label]
        y_true = input_true_value[preds_flat==label]
        y_id = input_id[preds_flat==label]
    
        print(f'Beruf: {input_val[preds_flat==label]}')
        print(f'pred: {getLabel(y_preds)}')
      
          




        result =  np.stack([y_beruf,getLabel(y_preds),y_true],axis=1)
        arr.append(result)
        result=[]
    return arr

In [ ]:
data = get_data_and_label(predictions,inputData)

In [ ]:
output_data = []
data=get_data_and_label(predictions,inputData)
endResult=[]
for x in data:
  for y in x:
    endResult.append(y.tolist())

#df = df.rename(columns={"0": "Berufe", "1": "True value", "2": "Prediction", "Berufsbezeichnung_1": "Berufsbezeichnung_True value", "Berufsbezeichnung_2": "Berufsbezeichnung_Prediction"})

    
dp = []    
dp = pd.DataFrame(endResult)
dp.columns=['occupation','predictions','true_value']
dp.columns.tolist()

dp.to_csv('result.csv')
from google.colab import files
files.download("result.csv")
